# Import packages

In [139]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import KFold, cross_validate, LeaveOneOut
import time

# Writing a function for our needs:
- ~~Handle importing a single or a list of dataframes~~ Handle this with a loop that incorporates the function
- Decide whether a dataframe needs to have the zero columns taken out
- Possibly allow for PCA inside of the function? TBD
- Return results to a dataframe, write them to csv
- Allow user to specfiy notes for the runs
- Specify models used for training
- Cross validation will be on by default, too complicated to have both functionality


In [140]:
def train_test_models(file_name, models='ALL', crossval_method='kfold',
                        zero_column_delete=True, absolute_value=True, notes=None):
    """[summary]

    Args:
        file_name ([type]): file to use for data
        models (str, optional): Model to be used for training and testing. Defaults to 'ALL'.
                                If all is selected, many different types of classifiers will
                                be used for training and testing.
        crossval_method (str, optional): Cross validation method used for model training.
                                         'kfold' and 'loo' are acceptable inputs.
                                         Defaults to 'kfold'.
        zero_column_delete (bool, optional): Gets rid of columns with all values=0. Defaults to True.
        absolute_value (bool, optional): Takes the absolute value of every datapoint. Defaults to True.
        notes ([type], optional): Description of the run. Defaults to None.
    """

    print("############################")
    print("Begining train_test function")
    print("############################")
    print(f'File the data is being pulled from: {file_name}')

    # Load in the data using pandas into a dataframe
    data_df = pd.read_csv(file_name, header=None)
    print(f'Data is of shape {data_df.shape}')

    # Delete the zero value columns if argument is provided
    if zero_column_delete:
        print('Columns with all Zeros are being removed')
        data_df = data_df.loc[:, (data_df != 0).any(axis=0)]
    
    # Get values from dataframe 
    X = data_df.iloc[:, 1:].values
    y = data_df.iloc[:, 0].values

    # Set absolute value of X if argument is provided
    if absolute_value:
        X = abs(X)
    
    # Initialize models if 'ALL' is selected
    if models == 'ALL':
        models = [AdaBoostClassifier(),
                GradientBoostingClassifier(),
                RandomForestClassifier(), 
                DecisionTreeClassifier(),
                ExtraTreesClassifier(),
                svm.SVC(kernel='rbf'),
                SGDClassifier(random_state=2021)]

    ## Check to see if model is in list or not
    elif type(models) is not list:
        print(f'{models} is not given in list format')
        raise

    # Initialize cross validation
    if crossval_method == 'kfold':
        cross_val = KFold(n_splits=10,
                          shuffle=True,
                          random_state = 42)
    elif crossval_method == 'loo':
        cross_val = LeaveOneOut()
    else:
        print('crossval_method neither kfold or loo')

    # Initiailze dataframe to store results
    results_df = pd.DataFrame([], columns = ['filename',
                                                'model',
                                                'train_scores',
                                                'test_scores',
                                                'min_test',
                                                'avg_test',
                                                'max_test',
                                                'notes'])
    
    # Model training testing loop
    for model in models:
        print('-----------------')
        print(f'{model} being used')
        results = cross_validate(model, X, y, cv=cross_val, return_train_score=True)
        train_score = results['train_score']
        test_score = results['test_score']
        test_min = min(test_score)
        test_avg = np.mean(test_score)
        test_max = max(test_score)
        print(f'train scores: {train_score}')
        print(f'test scores: {test_score}')
        ## Create a dict to store results (makes dataframes easier)
        results_dict = {'filename': file_name,
                        'model': str(model),
                        'train_scores': [train_score.round(3)],
                        'test_scores': [test_score.round(3)],
                        'min_test': test_min,
                        'avg_test': test_avg,
                        'max_test': test_max,
                        'notes': notes,
                        }
        temp_df = pd.DataFrame(results_dict, index=[0])
        results_df = results_df.append(temp_df, ignore_index=True)
        del results
        del model
        time.sleep(2)
        print('---------------------------------')

    return(results_df)

# Test function

In [141]:
# test_file = './data/eig_centrality.csv'
# results = train_test_models(test_file)

In [142]:
# results

# Question about how absolute value affects results

In [143]:
# # Create a dataframe to store all results from all tests
# master_results = pd.DataFrame(None)

# # Raw data with negatives and zeros
# notes = 'raw data, has negatives, has zeros'
# raw_data_results = train_test_models('./data/eig_centrality.csv', zero_column_delete=False, absolute_value=False,
#                                     notes=notes)

# # Raw data with negatives and zeros taken out
# notes = 'raw data, has negatives, zeros are removed'
# noZeros_Negatives_data_results = train_test_models('./data/eig_centrality.csv', zero_column_delete=True, absolute_value=False,
#                                     notes=notes)

# # Raw data with no negatives and no zeros
# notes = 'raw data, no negatives(absolute values only), zeros are removed'
# noZeros_noNegatives_raw_data_results = train_test_models('./data/eig_centrality.csv', zero_column_delete=True, absolute_value=True,
#                                     notes=notes)

# # Elasticnet results
# notes = 'Elasticnet Used as dimensionality reduction'
# notes = 'Dimensionality reduction using elasticnet'
# elastic_net_results = train_test_models('./data/60x61.csv', zero_column_delete=False, absolute_value=False,
#                                         notes=notes)

# master_results = master_results.append(raw_data_results, ignore_index=True)
# master_results = master_results.append(noZeros_Negatives_data_results, ignore_index=True)
# master_results = master_results.append(noZeros_noNegatives_raw_data_results, ignore_index=True)
# master_results = master_results.append(elastic_net_results, ignore_index=True)


In [144]:
# master_results.head(15)

In [145]:
# master_results.to_csv('models_results.csv',index=False)

# Multiple data files
Upload a zip file containing all your data
You can use the unix command line interface through the jupyter notebook by using **"!"**

In [146]:
# unzip data folder
'''
Use the folder icon on the left side to open the running directory.
You must be connected to a run time for the directory to be shown.
Before you run the following cells
'''
!unzip ./data.zip

unzip:  cannot find or open ./data.zip, ./data.zip.zip or ./data.zip.ZIP.


In [147]:
# use os to get files in the said directory
data_directory = './data' # Specify the data folder ## This is necessary ##
data_files = [] # create an empty list object to store the file paths
for x in os.listdir('./data'):
    if x.endswith('.csv'): # get only files that are csv
        data_files.append(f'{data_directory}/{x}') # get direct path
print(data_files)

['./data/60x61.csv', './data/eig_centrality.csv']


In [148]:
# loop over the files to train models and get results
master_df = pd.DataFrame(None) # create a dataframe to store model training results
for data_file in data_files:
    # Run our function uisng "loo" cross validation
    # If you want kfold change the crossval_method='kfold'
    run_results = train_test_models(data_file, models='ALL', crossval_method='loo')
    master_df = master_df.append(run_results, ignore_index=True)

############################
Begining train_test function
############################
File the data is being pulled from: ./data/60x61.csv
Data is of shape (60, 61)
Columns with all Zeros are being removed
-----------------
AdaBoostClassifier() being used
train scores: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
test scores: [1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.
 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.]
---------------------------------
-----------------
GradientBoostingClassifier() being used
train scores: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
test scores: [0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1.

In [149]:
# view results
master_df.head(40)

,filename,model,train_scores,test_scores,min_test,avg_test,max_test,notes
0,./data/60x61.csv,AdaBoostClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...",0.0,0.650000,1.0,None
1,./data/60x61.csv,GradientBoostingClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",0.0,0.466667,1.0,None
2,./data/60x61.csv,RandomForestClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...",0.0,0.666667,1.0,None
3,./data/60x61.csv,DecisionTreeClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...",0.0,0.583333,1.0,None
4,./data/60x61.csv,ExtraTreesClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.0,0.733333,1.0,None
5,./data/60x61.csv,SVC(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.0,0.900000,1.0,None
6,./data/60x61.csv,SGDClassifier(random_state=2021),"[1.0, 1.0, 0.492, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.0,0.900000,1.0,None
7,./data/eig_centrality.csv,AdaBoostClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, ...",0.0,0.433333,1.0,None
8,./data/eig_centrality.csv,GradientBoostingClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, ...",0.0,0.450000,1.0,None
9,./data/eig_centrality.csv,RandomForestClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...",0.0,0.450000,1.0,None


In [150]:
# Sort values by average test value
master_df.sort_values(by=['avg_test'],axis=0, ascending=False)

,filename,model,train_scores,test_scores,min_test,avg_test,max_test,notes
5,./data/60x61.csv,SVC(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.0,0.900000,1.0,None
6,./data/60x61.csv,SGDClassifier(random_state=2021),"[1.0, 1.0, 0.492, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.0,0.900000,1.0,None
4,./data/60x61.csv,ExtraTreesClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.0,0.733333,1.0,None
2,./data/60x61.csv,RandomForestClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...",0.0,0.666667,1.0,None
0,./data/60x61.csv,AdaBoostClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...",0.0,0.650000,1.0,None
3,./data/60x61.csv,DecisionTreeClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...",0.0,0.583333,1.0,None
12,./data/eig_centrality.csv,SVC(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...",0.0,0.500000,1.0,None
11,./data/eig_centrality.csv,ExtraTreesClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",0.0,0.483333,1.0,None
1,./data/60x61.csv,GradientBoostingClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",0.0,0.466667,1.0,None
8,./data/eig_centrality.csv,GradientBoostingClassifier(),"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, ...",0.0,0.450000,1.0,None


In [151]:
# Save the results to disk
master_df.to_csv('./all_data_files_results.csv')